# Step : 1 Create Training Data

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

#get the training data we previously made
data_path = './faces/user/'
onlyfiles = [ f for f in listdir(data_path) if isfile(join(data_path, f)) ]

#create arrays for training data and labels
Training_data , Labels = [] , []

#open training images in our data_path 
#create numpy array for training_data
for i , files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path , cv2.IMREAD_GRAYSCALE)
    Training_data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

#create a numpy array for both training data and labels 
Lables = np.asarray(Labels, dtype=np.int32)

#initialize facial recognizer
srushti_model  = cv2.face_LBPHFaceRecognizer.create()


#lets train our model
srushti_model.train(np.asarray(Training_data), np.asarray(Labels))
print('model trained successfuly')




# Step: 2  Train The Model

In [ ]:
import cv2
import numpy as np 

#load haar face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#load functions
def face_extractor(img):
    #functions detect faces and returns cropped faces
    #if no face detected it will return input image
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3 , 5)
    
    if faces is ():
        return None
    
    #crop all faces found
    for(x, y, w, h) in faces:
        cropped_faces = img[y:y+h, x:x+w]
        
    return cropped_faces

#initializes webcam
cap = cv2.VideoCapture(0)
count = 0

#collect 100 samples of your face from webcam input
while True:
    ret , frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        #save file in specific directory
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path , face)
        
        #put count on images and display live count 
        cv2.putText(face, str(count), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('face cropper', face)
    
    else:
        print('face not found')
        pass
    if cv2.waitKey(1) == 13 or count == 100:
        break
        
cap.release()
cv2.destroyAllWindows()
print('collecting samples complete')

        

# Step:3 Run Our Facial Recognition

In [ ]:
import cv2
import numpy as np
import os
import time

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_detector(img, size=0.5):
    #convert img to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3 , 5)
    if faces is():
        return img, []
    
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x,y), (x+w , y+h), (0,255,255), 2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
        return img, roi

#open webcam
cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        #pass face to prediction model 
        #'results' comprises of tuple containing the label and the confidence value
        results = srushti_model.predict(face)
        
        if results[1] < 500:
            confidence = int(100 *(1-(results[1])/400))
            display_string = str(confidence) + '%confident it is user'
        
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120, 150), 2)
        
        if confidence > 90:
            cv2.putText(image,'hey srushti', (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('face recognised', image)
            
            
            print("************************************launch ec2 instance********************************")
            os.system("aws ec2 run-instances --image-id  ami-0b2ca94b5b49e0132  --instance-type t2.micro  --key-name srushti")
            print("instance launched")
            
            
            print("************************************create volume********************************")
            os.system("aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone us-west-1b")
            print("volume created")
            
            #to attach instance and volume
            instance_id = input("Enter Instance ID : ")
            volume_id = input("Enter Volume ID : ")
            os.system("aws ec2 attach-volume --volume-id {} --instance-id {} --device /dev/sdf",format(volume_id,instance_id))
            break


            
            
        else:
            cv2.putText(image,'not recognized ', (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('face recognition', image)
            

    except:
        cv2.putText(image, 'no face found', (220, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(image,'where is srushti!!!', (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('face recognition', image)
        pass
    

    
    if cv2.waitKey(1) == 13: 
        break
        
cap.release()
cv2.destroyAllWindows()

       